# Webcrawling 

> ### ★ 크롤링 방법 3가지(우선순!)
#### 1. API사용
#### 2. HTML-beautiful_soup4
#### 3. Selenium 
   - 서버에서 돌릴때 두가지방법 : 크롬드라이버의 헤드리스사용, xufb 메모리상에 가상화면을 띄워서 사용

> #### 4. Scrapy (완전 웹 크롤링을 위해 만들어진 파이썬 모듈)

In [4]:
#셀레늄 선언 및 드라이버 설정
from selenium import webdriver

driver = webdriver.Chrome()

In [5]:
driver.get("http://www.google.com")

In [6]:
import requests

from bs4 import BeautifulSoup

In [1]:
import forecastio

ModuleNotFoundError: No module named 'forecastio'

## 목차

- api json을 사용 : 네이버 주식 데이터크롤링
- api json을 사용, forecastio : dark sky api로 날씨 데이터를 수집
- bs4 : 네이버 실시간 검색어 순위 , 다음 실시간 검색어 순위 크롤링
- web file url로 파일을 다운로드 받는 방법

# 네이버주식정보 가져오기

In [84]:
# 네이버주식정보 가져오기
import pandas as pd

from pandas.io.json import json_normalize

    # url제조함수
def make_url(pageSize=10, page=1):
    return 'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize='+str(pageSize)+'&page='+str(page)

    # 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() #json -> dict 형식으로 알아서 바꿔줌
    companys = json_info["result"]["itemList"] #list 형태로 바꿔줌
    a = json_normalize(companys)
    
    #행이름 바꾸기
    a.columns = ["주식수량","거래량","종목코드","등락율","전일비","시가총액","mt","종목","시세","전일가","rf"]

    # 일일이 열에 넣는방법은 아래와 같다    
#     df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
    
#     for company in companys:
#         df.loc[len(df)] = {
#             "종목":company["nm"],
#             "시세":company["nv"],
#             "전일비":company["cv"],
#             "등락율":company["cr"],
#             "시가총액":company["mks"],
#             "거래량":company["aq"]
#         }
    
    return a
    
    
#---함수를 써보자--------------------------    
url = make_url(5,1)
get_data(url)


,주식수량,거래량,종목코드,등락율,전일비,시가총액,mt,종목,시세,전일가,rf
0,808076,16839154,005930,-1.14,-550,3058808,0,삼성전자,47650,48200,5
1,255711,2936604,000660,0.81,700,633362,0,SK하이닉스,87000,86300,2
2,536055,1825018,068270,6.61,18500,374307,0,셀트리온,298500,280000,2
3,91322,2402876,005935,-2.45,-950,341572,0,삼성전자우,37800,38750,5
4,85210,232924,005490,-1.22,-4500,318232,0,POSCO,365000,369500,5


# 날씨데이터 가져오기
## dark sky api
- 위도와 경도를 입력하면 날씨기후정보를 상세히 알려주는 api
- https://darksky.net/dev/account - 내아이디 brightscannon@gmail
- get방식 포맷 = https://api.darksky.net/forecast/[key]/[latitude],[longitude]
- json 형태로 파싱하여 사용가능


In [19]:
FORECAST_TOKEN = '809c952e330b1e5d704de2851f7d8ac1'

In [86]:
# darksky데이터 api로 가져오기
def forecast(lat, lng):
    url = 'https://api.darksky.net/forecast/{}/{},{}'.format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    
    return json_info['timezone'], json_info['hourly']['summary']
    
    
#------이제 함수를 써보자!!!------------    
lat = 37.2
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy starting this evening.')

In [26]:
# forecastio 사용해서 정보얻기

def forecast2(lat, lng):
    forecast = forecastio.load_forecast()
    byHourly = forecast.hourly()
    return byHourly.summary

#------이제 함수를 써보자!!!------------    
lat = 37.2
lng = 126.954
forecast2(lat, lng)

NameError: name 'forecastio' is not defined

## bs4 실시간 급상승 키워드 크롤링~!
- 네이버 급상승키워드 크롤링
- 다음 급상승키워드 크롤링

In [79]:
#네이버 급상승검색어 크롤링하기

#dom.select - 여러개의 html 엘리먼트를 셀렉팅할때 사용, 결과로 리스트 데이터를 리턴
#dom.select_one - 하나의 html 엘리먼트를 셀렉팅할때 사용, 결과로 문자열데이터를 리턴

def naver():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content, "html.parser")
    
    #1차 거르기
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")
#     print(len(keywords)) # 먼저 갯수체크^^
    
    #2차 걸러서 정리
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text
        }
    
    return df

#----이게 함수를 써보자~!

naver()

,rank,keyword
0,1,이집트 우루과이
1,2,포항 약국
2,3,블랙핑크
3,4,뮤직뱅크
4,5,살라
5,6,러시아 사우디 하이라이트
6,7,최원명
7,8,모로코 이란
8,9,쌈디
9,10,강혁민


In [47]:
import time
ls = []
for idx in range(3):
    print(idx)
    ls.append(naver())
    time.sleep(1)

0
1
2


In [51]:
# 다음의 실시간 이슈 검색어 10개 수집
# 네이버 20개 데이터와 다음 10개 데이터에서 중복되는 데이터를 출력

In [61]:
def daum():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, "html.parser")
    
    #1차 거르기
    keywords = dom.select(".list_hotissue.issue_row")
    print(len(keywords)) # 먼저 갯수체크^^
    print(keywords) # 내용체크^^
    
    #2차 걸러서 정리
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".link_iss").text,
            "keyword":keyword.select_one(".link_issue:nth_child(2)").text
        }
    
    return df

#----이게 함수를 써보자~!

daum()

3
[<ol class="list_hotissue issue_row">
<li class="current">
<div class="roll_txt @1">
<div class="rank_cont">
<span class="num_pctop rank1"><span class="ir_wa">1위</span></span>
<span class="txt_issue"><a class="link_issue" href="https://search.daum.net/search?w=tot&amp;q=%EB%AF%BC%EA%B0%91%EB%A3%A1&amp;DA=ATG&amp;rtmaxcoll=1TH">민갑룡</a></span>
</div>
<div aria-hidden="true" class="rank_cont">
<span class="num_pctop rank1"><span class="ir_wa">1위}</span></span>
<span class="txt_issue"><a class="link_issue" href="https://search.daum.net/search?w=tot&amp;q=%EB%AF%BC%EA%B0%91%EB%A3%A1&amp;DA=ATG&amp;rtmaxcoll=1TH" tabindex="-1">민갑룡</a></span>
</div>
</div>
</li>
<li class="next">
<div class="roll_txt @2">
<div class="rank_cont">
<span class="num_pctop rank2"><span class="ir_wa">2위</span></span>
<span class="txt_issue"><a class="link_issue" href="https://search.daum.net/search?w=tot&amp;q=%EA%B2%BD%EC%B0%B0%EC%B2%AD%EC%9E%A5&amp;DA=ATG&amp;rtmaxcoll=1TH">경찰청장</a></span>
</div>
<div aria-hidd

AttributeError: 'NoneType' object has no attribute 'text'

In [78]:
def daum():
    df = pd.DataFrame()
    rank = []
    keywords = []
    response = requests.get("https://www.daum.net")
    dom = BeautifulSoup(response.content,'html.parser')
    keywords = dom.select('.realtime_part > .list_hotissue.issue_row > li')
    print(len(keywords))
    
    rank = [keyword.select_one(".ir_wa").text[:-1] for keyword in keywords]
    keywords = [keyword.select_one(".link_issue").text for keyword in keywords]
    
    df["rank"]= rank
    df["keyword"]= keywords
    
    return df

daum()

10


,rank,keyword
0,1,국가장학금
1,2,민갑룡
2,3,경찰청장
3,4,블랙핑크
4,5,한국장학재단
5,6,조현배
6,7,안희정
7,8,심영은
8,9,인랑
9,10,장윤정


In [77]:
# 두 사이트 키워드 비교하기

naver_df = naver()
daum_df = daum()


result = [keyword for keyword in naver_df["keyword"]
          if daum_df["keyword"].str.contains('keyword').any()]

result

10


[]

### file download : requests
- file의 url을 이용해서 파일 다운로드

In [83]:
#원래 url이 너무 길기때문에 bitly에서짧게 줄여주는 서비스를 이용해본다.
url = "https://r5---sn-p5qs7n7l.googlevideo.com/videoplayback?c=WEB&mime=video%2Fmp4&id=o-AFXHiKDdzivI5SAdSVkrIGYgNSzGpWT3mqFDfBxbbGb-&mn=sn-p5qs7n7l%2Csn-qxoedn7d&mm=31%2C26&signature=06036B34CF919079990002BF798B109E5ADD4C60.E14DD41594388C5A5DDD6C03F7F159F9FF849078&ms=au%2Conr&fvip=5&mv=u&source=youtube&dur=38.103&lmt=1528961448486107&itag=22&key=yt6&ip=23.94.22.66&ei=P18jW4S0EZaU8wTo7ZOgCQ&ipbits=0&sparams=dur%2Cei%2Cid%2Cip%2Cipbits%2Citag%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Cratebypass%2Crequiressl%2Csource%2Cexpire&pl=24&ratebypass=yes&expire=1529066399&mt=1529043903&requiressl=yes&video_id=dnGs2eOE6hQ&title=How+to+shoot+Soccer+on+iPhone+X+%E2%80%94+Apple"


#청크를 사용하여 다운로드
def download(title, download_link):
    response = requests.get(download_link, stream = True)
    file_size = 0
    with open(title, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): # byte
            if chunk:
                file_size += 1024
                f.write(chunk)
    return file_size


# bitly에서 url줄였음
download_link= "https://bit.ly/2JKy2nD"
title = "video.mp4"

file_size = download(title, download_link)

str(round(file_size/1024/1024,2)) + "MB" #용량이 바이트로 나오기때문에 메가로 변환하여 보여줌

'4.76MB'